In [ ]:
import tensorflow as tf
import numpy as np
import rocket_tools as rt
from keras import backend 
from keras.backend import learning_phase
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"  

In [ ]:
image_size = 128
n_images = 3
steps = 1000

tf.reset_default_graph()

x_input = tf.placeholder(tf.float32,
                         [None, image_size, image_size, n_images],
                         name='input')


model = rt.model.U_NET(input_=(image_size, image_size, n_images))
y_pred = model(x_input)

sess = tf.InteractiveSession()
saver = tf.train.Saver()

MODEL_PATH = './../saved_models/u_net_image_size=128_snr_all_w_more_dot'
saver.restore(sess, MODEL_PATH)

results = {}
snr_test = np.linspace(0.1, 10.0, 100)

for snr in snr_test:
    predictions = []
    true_masks = []

    image_generator = rt.image_gen.ImageGenerator(
        image_size=(image_size, image_size))
    batch_generator = rt.batch_generator.BatchGenerator(image_generator, batch_size,
                                                        snr_mean=snr, snr_std=0.0, n_images=n_images)
    for step in range(steps):
        x_array, y_true_array = next(batch_generator)
        y_pred_array = sess.run(y_pred, feed_dict={
            x_input: x_array,
            learning_phase(): 0
        })
        predictions.append(y_pred_array)
        true_masks.append(y_true_array)

    results['snr_{}'.format(snr)] = {
        'true': true_masks,
        'pred': predictions
    }

np.savez_compressed('resulst_snr_0.1_10_100.npz', results=results)
